<a href="https://colab.research.google.com/github/izaleme/CienciaDeDados/blob/main/Classificador_de_Spam_com_Naive_Bayes_e_SKLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Alunos: Izabela Leme, Rhainan Domingos***

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/')

import numpy
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import os

NEWLINE = '\n'
SKIP_FILES = set(['cmds'])

In [3]:
def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
    for file_name in file_names:
        if file_name not in SKIP_FILES:
            file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):
            past_header, lines = False, []
            #f = open(file_path)
            f = open(file_path, mode="r", encoding="Latin-1")
            for line in f:
                if past_header:
                    lines.append(line)
                elif line == NEWLINE:
                    past_header = True
            f.close()
            yield file_path, NEWLINE.join(lines)#.decode('cp1252', 'ignore')

In [4]:
from pandas import DataFrame

def build_data_frame(path, classification):
  data_frame = DataFrame({'text': [], 'class': []})
  for file_name, text in read_files(path):
    data_frame = data_frame.append(
        DataFrame({'text': [text], 'class': [classification]}, index=[file_name]))
  return data_frame

In [6]:
HAM = 0
SPAM = 1

SOURCES = [
    ('/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/spam',        SPAM),
    ('/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/easy_ham',    HAM),
    ('/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/hard_ham',    HAM)
    ]

data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
  data = data.append(build_data_frame(path, classification))
  
data = data.reindex(numpy.random.permutation(data.index))

In [7]:
count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(numpy.asarray(data['text']))

In [8]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = numpy.asarray(data['class'])
classifier.fit(counts, targets)

MultinomialNB()

In [9]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions # [1, 0]

array([1., 0.])

In [10]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
  ('vectorizer',  CountVectorizer()),
  ('classifier',  MultinomialNB()) ])

pipeline.fit(numpy.asarray(data['text']), numpy.asarray(data['class']))
pipeline.predict(examples) # [1, 0]

array([1., 0.])

In [11]:
from sklearn.naive_bayes import BernoulliNB

In [12]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('classifier',         BernoulliNB(binarize=0.0)) ])

In [13]:
data

,text,class
/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/spam/0403.5aa6261d36d1362bcd181ed7738de7f7,1) Lose 22.5lbs in 3 weeks!\n\nFlush Fat Away ...,1.0
/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/spam/0394.9c882c72ddfd810b56776fdaa1c727a6,<html>\n\n<head>\n\n<title>FREE* Liz Claiborne...,1.0
/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/hard_ham/0208.df5d7a4e950a3aa80a71fbb5186b0f87,\n\nThis is a multi-part message in MIME forma...,0.0
/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/easy_ham/0007.c75188382f64b090022fa3b095b020b0,\n\n> I just had to jump in here as Carbonara ...,0.0
/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/spam/0043.8d93819b95ff90bf2e2b141c2909bfc9,--TEP-2108411027.1463792894.1027038601\n\nCont...,1.0
...,...,...
/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/easy_ham/0295.27b3176967da2ccf237679a15788d821,Bill:\n\n> I've decided that I ought to put so...,0.0
/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/easy_ham/0342.16612b49bb48a41b6bdba8dff6bb2399,I wrote:\n\n> [The bytecode interpreter] may i...,0.0
/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/hard_ham/0166.4b3874844ccb78cd7ca2cd03876124b0,<html><head><!--\n\n**************************...,0.0
/content/drive/MyDrive/Colab Notebooks/Datas/spamassassin/easy_ham/0661.123776ef7c55f1119daa8f77075f77cb,">>>>> ""J"" == Justin Mason <jm@jmason.org> writ...",0.0
